<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Processamento-Colunar-de-Dados" data-toc-modified-id="Processamento-Colunar-de-Dados-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Processamento Colunar de Dados</a></span><ul class="toc-item"><li><span><a href="#Motivação" data-toc-modified-id="Motivação-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Motivação</a></span></li><li><span><a href="#So-far-..." data-toc-modified-id="So-far-...-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>So far ...</a></span></li><li><span><a href="#O-que-aconteceu?" data-toc-modified-id="O-que-aconteceu?-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>O que aconteceu?</a></span></li></ul></li><li><span><a href="#Columnar-storage" data-toc-modified-id="Columnar-storage-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Columnar storage</a></span></li><li><span><a href="#Frameworks" data-toc-modified-id="Frameworks-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Frameworks</a></span><ul class="toc-item"><li><span><a href="#pandas" data-toc-modified-id="pandas-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>pandas</a></span></li><li><span><a href="#dask" data-toc-modified-id="dask-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>dask</a></span></li><li><span><a href="#numba" data-toc-modified-id="numba-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>numba</a></span></li></ul></li><li><span><a href="#Referências" data-toc-modified-id="Referências-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Referências</a></span></li></ul></div>

# Processamento Colunar de Dados

IF706 - Introdução à Ciência de Dados

Guilherme Peixoto, MSc. candidate, Data Scientist @ In Loco

---

INB4: Que setup você está usando?

Eu uso `jupyter notebook` com as extensões `Table of Contents (2)` e `Collapsible Headings`. Instruções no final.

## Motivação

**Expectativa**: 
* gargalo do meu tempo como data scientist é melhorando meu modelo

**Realidade**: 
* 80% cleaning data, 10% waiting, 5% making coffee, 5% modeling

Precisamos melhorar a eficiência reduzindo nosso tempo idle

**Casos de uso**

* Estatísticas descritivas
* Aplicar a mesma transformação ao longo de uma coluna
* Feature engineering
* Realizar operações “mundanas” rapidamente: preencher dados faltando, criar slices baseados no valor de outra coluna, etc.

**Objetivo** : “bater no disco” é caro, IO é gargalo e deve ser minimizado

## So far ...

Na graduação, aprendemos até agora a processar o dado um por um conforme ele vem, maratona style

Como seria o mundo de análise de dados sem processamento orientado a colunas?

In [1]:
from sklearn.datasets import california_housing

In [2]:
data = california_housing.fetch_california_housing()
data.keys()

dict_keys(['data', 'target', 'feature_names', 'DESCR'])

In [3]:
print(data["DESCR"])

.. _california_housing_dataset:

California Housing dataset
--------------------------

**Data Set Characteristics:**

    :Number of Instances: 20640

    :Number of Attributes: 8 numeric, predictive attributes and the target

    :Attribute Information:
        - MedInc        median income in block
        - HouseAge      median house age in block
        - AveRooms      average number of rooms
        - AveBedrms     average number of bedrooms
        - Population    block population
        - AveOccup      average house occupancy
        - Latitude      house block latitude
        - Longitude     house block longitude

    :Missing Attribute Values: None

This dataset was obtained from the StatLib repository.
http://lib.stat.cmu.edu/datasets/

The target variable is the median house value for California districts.

This dataset was derived from the 1990 U.S. census, using one row per census
block group. A block group is the smallest geographical unit for which the U.S.
Census Bur

**Pergunta** : Qual a média da média de número de quartos na California?

In [4]:
accumulator = 0
for record in data["data"]:
    accumulator += record[2]
accumulator /= len(data["data"])
print(f"The mean of mean number of bedrooms in california is {accumulator:.3f}")

The mean of mean number of bedrooms in california is 5.429


## O que aconteceu?

No loop `for record in data["data"]`, precisamos "observar" o "record" (ou a linha) completa de dados, mas perceba que estamos usando uma única coluna em `record[2]`.

**Problemas**

* Ao invés de processarmos somente o que queríamos, desnecessariamente manipulamos todas as features
* Perda de produtividade
* Impossível de escalar -- OK nesse exemplo, mas imagine 15TB / dia

# Columnar storage

> A column-oriented DBMS (or columnar database management system) is a database management system (DBMS) that stores data tables by column rather than by row.
(Wikipedia : [Column-oriented DBMS](https://en.wikipedia.org/wiki/Column-oriented_DBMS))

Arquivo é armazenado **uma coluna por vez** e possui um esquema fortemente "tipado".

**Ganhos:**
* Maior compressão (teoria de Shannon), menos espaço em disco
* Menos leituras no disco; logo menos tempo idle aguardando IO (blocking)
* Evolução de esquema

Exemplo:
![](table.png)

Como é armazenado em disco:

![](disk.png)

**Como obter todos os dados para o item #6?**
* “Abre" apenas o offset de cada coluna

**Como obter o total de descontos emitidos?**
* Soma apenas a coluna de “descontos”

Na prática, várias otimizações existem, eg: agregações pré-computadas no header do arquivo. Isso é implementado em alguns formatos de arquivo colunares, o principal deles sendo `parquet`

# Frameworks

## pandas

Framework para manipulação de dados tabulares em memória

* **Pros**: API extremamente extensiva (inspirada nos dataframes nativos de R)
* **Cons**: single-thread

Leitura recomendada: [A Beginner’s Guide to Optimizing Pandas Code for Speed](https://engineering.upside.com/a-beginners-guide-to-optimizing-pandas-code-for-speed-c09ef2c6a4d6?gi=fe5489bbb068)

> *Vectorization* is the process of executing operations on entire arrays. \[...\] Pandas includes a generous collection of vectorized functions for everything from mathematical operations to aggregations and string functions (for an extensive list of available functions, check out the Pandas docs).

Operações podem ser feitas "de uma vez só" numa coluna inteira ao invés de ir linha por linha. `pandas` implementa diversas otimizações por debaixo dos panos, de forma que operações como adicionar uma constante para todos valores de uma coluna é feita de forma *quasi*-simultânea

*obs*: Caso não tenha `pandas` instalado, rodar `pip install --upgrade pandas` 

In [5]:
import pandas as pd
df = pd.DataFrame(data["data"], columns=data["feature_names"])
df.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25


Voltando à pergunta anterior: qual a média da média de número de quartos por bloco na California?

In [6]:
mean = df.AveRooms.sum() / len(df)
print(f"The mean of mean number of bedrooms in california is {accumulator:.3f}")

The mean of mean number of bedrooms in california is 5.429


Importante sempre usar a documentação de `pandas` como livro de cabeceira; existem *muitas* funções prontas e API é bastante extensa. Podemos resolver usando apenas `mean` diretamente:

In [7]:
mean = df.AveRooms.mean()
print(f"The mean of mean number of bedrooms in california is {mean:.3f}")

The mean of mean number of bedrooms in california is 5.429


Qual o percentil 75 (ou terceiro quartil) da população?

In [8]:
p75 = df.Population.quantile(q=0.75)
print(f"The third quartile is {p75:.0f}")

The third quartile is 1725


Como obter estatísticas descritivas de todas as colunas numéricas?

In [9]:
df.describe()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
count,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000
mean,3.870671,28.639486,5.429000,1.096675,1425.476744,3.070655,35.631861,-119.569704
std,1.899822,12.585558,2.474173,0.473911,1132.462122,10.386050,2.135952,2.003532
min,0.499900,1.000000,0.846154,0.333333,3.000000,0.692308,32.540000,-124.350000
25%,2.563400,18.000000,4.440716,1.006079,787.000000,2.429741,33.930000,-121.800000
50%,3.534800,29.000000,5.229129,1.048780,1166.000000,2.818116,34.260000,-118.490000
75%,4.743250,37.000000,6.052381,1.099526,1725.000000,3.282261,37.710000,-118.010000
max,15.000100,52.000000,141.909091,34.066667,35682.000000,1243.333333,41.950000,-114.310000


*"Avançado"*: qual o índice de correlação entre os pares de features?

Nesse caso, é calculado o índice de Pearson para todo par de features. Rápido lembrete que:

* $\rho_{x, y} \approx -1$ : forte correlação linear negativa
* $\rho_{x, y} \approx 0$ : sem correlação
* $\rho_{x, y} \approx 1$ : forte correlação linear positiva

In [10]:
df.corr()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
MedInc,1.000000,-0.119034,0.326895,-0.062040,0.004834,0.018766,-0.079809,-0.015176
HouseAge,-0.119034,1.000000,-0.153277,-0.077747,-0.296244,0.013191,0.011173,-0.108197
AveRooms,0.326895,-0.153277,1.000000,0.847621,-0.072213,-0.004852,0.106389,-0.027540
AveBedrms,-0.062040,-0.077747,0.847621,1.000000,-0.066197,-0.006181,0.069721,0.013344
Population,0.004834,-0.296244,-0.072213,-0.066197,1.000000,0.069863,-0.108785,0.099773
AveOccup,0.018766,0.013191,-0.004852,-0.006181,0.069863,1.000000,0.002366,0.002476
Latitude,-0.079809,0.011173,0.106389,0.069721,-0.108785,0.002366,1.000000,-0.924664
Longitude,-0.015176,-0.108197,-0.027540,0.013344,0.099773,0.002476,-0.924664,1.000000


Função `apply`: aplica uma função de alta ordem a uma coluna inteira. Necessário modificar a coluna de origem novamente.

Ex: passando a variável `HouseAge` para escala logarítimica em base neperiana:

In [11]:
from math import log
df["HouseAgeLog"] = df.HouseAge.apply(log)
df.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,HouseAgeLog
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,3.713572
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,3.044522
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,3.951244
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,3.951244
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,3.951244


Importantíssimo: `slice`s são "fatias" do conjunto, utilizadas a partir de séries booleanas. Permitem filtrar dados eficientemente.

Exemplo: 5 primeiros blocos com ao número médio de cômodos acima de `9`:

In [12]:
df[df.AveRooms > 9].head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,HouseAgeLog
512,12.2138,52.0,9.210227,1.039773,1001.0,2.843750,37.82,-122.23,3.951244
514,12.3804,52.0,9.122715,1.033943,1192.0,3.112272,37.82,-122.23,3.951244
1023,2.2417,15.0,10.515306,2.372449,573.0,2.923469,38.72,-119.93,2.708050
1024,3.1500,16.0,29.852941,5.323529,202.0,1.980392,38.52,-120.00,2.772589
1030,3.0125,15.0,10.168591,2.057737,1103.0,2.547344,38.55,-120.25,2.708050


Exercícios:

1. Qual o centróide geográfico do dataset? (uma linha!)
2. Quantos blocos existem com idade média da casa abaixo de 10 anos e com número de habitantes médio abaixo de 3 pessoas?
3. Dentre os blocos 10% mais populosos, qual a média da ordem de grandeza (potência de 10) da população?

**Exercício avançado:**

O [interquartile range](https://en.wikipedia.org/wiki/Interquartile_range) é definido como a diferença entre o terceiro quartil (Q3) e o primeiro (Q1). Um método simples para identificação de outliers é considerar que, para uma distribuição univariada, todas as observações que estão abaixo de $Q1 - 1.5IQR$ ou acima de $Q3 + 1.5IQR$ são considerados outliers.

Implemente o seguinte método `remove_outliers_iqr` que, dado um dataframe e o nome de uma coluna, retorna um *slice* daquele dataframe sem outliers, baseado no método IQR em cima da coluna fornecida. 

```python
def remove_outliers_iqr(df: pd.DataFrame, col_name: str) -> pd.DataFrame:
    raise NotImplementedError
```

** Outros métodos importantes **:
* `fillna`, `isnull`, `replace`

**Before you go ...**

## dask

> Dask is a flexible library for parallel computing in Python.

`dask` é uma alternativa multi-core e/ou distribuída para pandas.
* **Pros** : multi-core, cluster mode, escala
* **Cons**: API limitada, “pensar distribuído”, nem toda operação é trivialmente paralelizada

`dask` segue um modelo *lazy* computacional, então a operação não é de fato executada até que seja chamado `.compute()` explicitamente, retornando um dataframe `pandas` em memória.

In [13]:
import dask.dataframe as dd
from multiprocessing import cpu_count

N_CORES = cpu_count()
print(f"The machine running this notebook has {N_CORES} cpus.")

ddf = dd.from_pandas(df, npartitions=N_CORES)
mean = ddf["AveRooms"].mean().compute()
print(f"The mean of mean number of bedrooms in california is {mean:.3f}")

The machine running this notebook has 4 cpus.
The mean of mean number of bedrooms in california is 5.429


Quando usar `dask` ao invés de `pandas`?

1. dataset não couber em memória;
2. `pandas` estiver proibitivamente lento e seja desejado usar todos os cores da máquina para paralelizar o processamento;
3. dataset estiver quebrado em inúmeros arquivos - relacionado a (1)

## numba

> Numba is an open source JIT compiler that translates a subset of Python and NumPy code into fast machine code. Numba translates Python functions to optimized machine code at runtime using the industry-standard LLVM compiler library. Numba-compiled numerical algorithms in Python can approach the speeds of C or FORTRAN.

TLDR : operações matriciais *ultra fast*

In [14]:
from numba import jit

@jit(nopython=True)
def add_five(series):
    return series + 5

In [15]:
print(f"Timing `add_five` with numba:")
%timeit add_five(df.HouseAge.values)
print(f"\nTiming native pandas vectorized addition")
%timeit df.HouseAge + 5

Timing `add_five` with numba:
The slowest run took 6.36 times longer than the fastest. This could mean that an intermediate result is being cached.
47.3 µs ± 43.7 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)

Timing native pandas vectorized addition
159 µs ± 11.4 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


Observações:
1. `numba` opera em cima diretamente de `numpy` arrays, e não de `Series` de `pandas`;
2. A anotação `nopython=True` está dizendo "compile estritamente em C, sem usar Python nativamente". Na prática, sem essa anotação, `numba` não irá obter ganhos de performance.
3. Na prática, usado só em casos que o código precisa ser de alta performance. Há um sacrifício de produtividade e ergonomia de parte do desenvolvedor, e não é trivial implementar operações vetorizadas

Exemplo: calcule a distância **em KM** de todos os pontos do dataset para o centróide

In [16]:
import numpy as np

@jit(nopython=True)
def haversine_numba(lon1, lat1, lon2, lat2):
    R = 6371.0  # radius of the earth in **KM**
    lon1 = np.radians(lon1)
    lon2 = np.radians(lon2)
    lat1 = np.radians(lat1)
    lat2 = np.radians(lat2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2
    c = 2 * np.arcsin(np.sqrt(a))
    return R * c

In [17]:
%%time
print(f"Calculating time to run without numba...")
center_lat, center_lon = df[["Latitude", "Longitude"]].mean()
dist = haversine_numba(df.Longitude.values, df.Latitude.values, center_lon, center_lat)

Calculating time to run without numba...
CPU times: user 483 ms, sys: 11.6 ms, total: 495 ms
Wall time: 500 ms


# Referências

* Docs
    * https://pandas.pydata.org/pandas-docs/stable/
    * http://docs.dask.org/en/latest/
    * https://numba.pydata.org


* Designing Data-Intensive Applications
    * https://dataintensive.net 
    * https://www.amazon.com/Designing-Data-Intensive-Applications-Reliable-Maintainable/dp/1449373321


* Extensões do Jupyter notebook: 
    * https://jupyter-contrib-nbextensions.readthedocs.io/en/latest/nbextensions.html
    * https://jupyter-contrib-nbextensions.readthedocs.io/en/latest/install.html
    * https://towardsdatascience.com/jupyter-notebook-extensions-517fa69d2231
    

* Misc
    * https://en.wikipedia.org/wiki/Interquartile_range
    * https://en.wikipedia.org/wiki/Pearson_correlation_coefficient
    * https://parquet.apache.org